In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
engine = create_engine('postgresql://postgres:qwerty111!@@localhost/moodle')

In [3]:
class DB_in:
    """This class has methods that get various information on our database,regarding the tables"""
    
    def __init__(self, engine):
        self.engine = engine
    
    def table_count(self,query = "SELECT COUNT(*)FROM information_schema.tables"):
        df = pd.read_sql(query,self.engine)
        return df
    
    def row_per_table(self,table):
        query = "SELECT COUNT(*) FROM {table}".format(table=table)
        df = pd.read_sql(query,self.engine)
        print (df)
        
    """    def row_per_table_multi(self,table_list):
        query = "SELECT COUNT(*) FROM {table}".format(table=table)
        for i in table_list:
            """
    def read_table(self,table):
        query = "SELECT * FROM {table}".format(table=table)
        table_data = pd.read_sql(query,self.engine)
        return table_data
            

In [35]:
class DB_fetch:
    """This class has methods that fetch specific information from the connected database"""
    from sqlalchemy import create_engine
    import pandas as pd

    def __init__(self, engine):
        self.engine = engine
    
    def total_grade(self, query):
        """ This method returns the overall grade for each learner"""
        df = pd.read_sql(query, self.engine)
        df2 = df[['userid', 'itemname', 'finalgrade']]
        df2.sort_values('userid', inplace=True)
        return df2
        
        
    def forum_post_count(self, query):
        """ This method returns the number of posts a learner contributed to the forum"""
        df = pd.read_sql(query, self.engine)
        df2 = df.loc[(df.mailed == 1) & (df.deleted == 0)]
        return df2.shape[0]

In [4]:
class Data_extraction:
    """ This class extracts various information from our db table"""
    def __init__(self,engine):
        self.engine = engine
    
    def dedication_time_user(self,log_query = "SELECT * FROM mdl_logstore_standard_log"):
        df = pd.read_sql(log_query,self.engine)
        user_id =  df.userid.unique()
        d_times = {}
        for user in user_id:
            log_user = df[df.userid == user].sort_values('timecreated')

            d_time = 0 
            isLoggedIn = 0
            loggedIn_timecreated = 0

            for i in range(len(log_user)): 
                row = log_user.iloc[i,]

                row_next = log_user.iloc[i+1,] if i+1 < len(log_user) else row

                if(row.action == "loggedin"): 
                    isLoggedIn = 1
                    loggedIn_timecreated = row.timecreated

                if( (i+1 == len(log_user)) | ( (row_next.action == "loggedin") & (isLoggedIn == 1) ) ):
                    d_time += row.timecreated - loggedIn_timecreated
                    isLoggedIn = 0

            d_times[user] = d_time
        dedication_time_df = pd.DataFrame({'userid':list(d_times.keys()),
                                  'dedication_time':list(d_times.values())})
        return dedication_time_df
    
    def activity_count_user(self,log_query = "SELECT * FROM mdl_logstore_standard_log"):
        df = pd.read_sql(log_query,self.engine)
        activity_log = df[['userid', 'action']]
        activity_log_by_user = activity_log.groupby('userid').count().sort_values('action', ascending=False)
        return activity_log_by_user
        
    def login_count_user(self,log_query = "SELECT * FROM mdl_logstore_standard_log"):
        df = pd.read_sql(log_query,self.engine)
        log_df_logged_in = df[df.action == 'loggedin'][['userid', 'action']]
        login_by_user = log_df_logged_in.groupby('userid').count().sort_values('action', ascending=False)
        return login_by_user
    

In [5]:
def top_x(df, percent):
    total_len = df.shape[0]
    top = int((total_len * percent)/100)
    return df.iloc[:top,]

In [6]:
data = Data_extraction(engine)

In [7]:
ded = data.dedication_time_user()


In [8]:
ded.head()

,dedication_time,userid
0,13335692,150
1,13134401,255
2,11909404,272
3,4894270,254
4,13074412,108


In [9]:
activity= data.activity_count_user()
activity.head()

,action
userid,
2,45023
246,13917
3,12922
917,10696
581,10533


In [12]:
login = data.login_count_user()
login.head()

,action
userid,
2,169
246,113
3,107
369,100
165,91


In [13]:
data = DB_fetch(engine)
table_count = data.table_count()

In [14]:
log_data = data.read_table("mdl_logstore_standard_log")
log_data.head()

,id,eventname,component,action,target,objecttable,objectid,crud,edulevel,contextid,...,contextinstanceid,userid,courseid,relateduserid,anonymous,other,timecreated,origin,ip,realuserid
0,25745,\core\event\notification_viewed,core,viewed,notification,notifications,2343.0,u,0,1011,...,150,150,0,255.0,0,N;,1550753076,web,197.237.117.85,NaN
1,25746,\core\event\notification_sent,core,sent,notification,notifications,2343.0,c,0,1,...,0,255,0,150.0,0,"a:1:{s:8:""courseid"";s:2:""46"";}",1550753076,web,197.237.117.85,NaN
2,25747,\core\event\notification_viewed,core,viewed,notification,notifications,2344.0,u,0,1133,...,272,272,0,255.0,0,N;,1550753076,web,197.237.117.85,NaN
3,25748,\core\event\notification_sent,core,sent,notification,notifications,2344.0,c,0,1,...,0,255,0,272.0,0,"a:1:{s:8:""courseid"";s:2:""46"";}",1550753076,web,197.237.117.85,NaN
4,25749,\core\event\notification_viewed,core,viewed,notification,notifications,2345.0,u,0,1115,...,254,254,0,255.0,0,N;,1550753076,web,197.237.117.85,NaN


In [15]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417554 entries, 0 to 417553
Data columns (total 21 columns):
id                   417554 non-null int64
eventname            417554 non-null object
component            417554 non-null object
action               417554 non-null object
target               417554 non-null object
objecttable          362981 non-null object
objectid             362981 non-null float64
crud                 417554 non-null object
edulevel             417554 non-null int64
contextid            417554 non-null int64
contextlevel         417554 non-null int64
contextinstanceid    417554 non-null int64
userid               417554 non-null int64
courseid             417554 non-null int64
relateduserid        325198 non-null float64
anonymous            417554 non-null int64
other                417554 non-null object
timecreated          417554 non-null int64
origin               417554 non-null object
ip                   328041 non-null object
realuserid      

In [16]:
context = data.read_table("mdl_context")
context.head()

,id,contextlevel,instanceid,path,depth,locked
0,1,10,0,/1,1,0
1,496,70,226,/1/72/430/496,4,0
2,2,50,1,/1/2,2,0
3,3,40,1,/1/3,2,0
4,499,70,228,/1/72/498/499,4,0


In [17]:
context.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4359 entries, 0 to 4358
Data columns (total 6 columns):
id              4359 non-null int64
contextlevel    4359 non-null int64
instanceid      4359 non-null int64
path            4359 non-null object
depth           4359 non-null int64
locked          4359 non-null int64
dtypes: int64(5), object(1)
memory usage: 204.4+ KB


In [18]:
user_data = data.read_table("mdl_user")
user_data.head()

,id,confirmed,policyagreed,deleted,suspended,idnumber,country,lang,firstaccess,lastaccess,lastlogin,currentlogin,lastip,gender,CountryCode
0,1,1,0,0,0,,Germany,en,0,0,0,0,,Male,DE
1,20,1,0,0,0,,Germany,en,1549745767,1562935333,1549745767,1562935333,197.215.32.111,Male,DE
2,15,1,0,0,0,,Germany,en,1549633236,1563537002,1563469481,1563537002,41.184.239.200,Male,DE
3,36,1,0,0,0,,Ghana,en,1549641288,1559227964,1549651186,1559227892,154.160.1.224,Male,GH
4,44,1,0,0,0,,Ghana,en,1549637001,1560095090,1557475553,1560094908,129.122.1.14,Male,GH


In [19]:
user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1052 entries, 0 to 1051
Data columns (total 15 columns):
id              1052 non-null int64
confirmed       1052 non-null int64
policyagreed    1052 non-null int64
deleted         1052 non-null int64
suspended       1052 non-null int64
idnumber        1052 non-null object
country         1052 non-null object
lang            1052 non-null object
firstaccess     1052 non-null int64
lastaccess      1052 non-null int64
lastlogin       1052 non-null int64
currentlogin    1052 non-null int64
lastip          1052 non-null object
gender          1052 non-null object
CountryCode     1052 non-null object
dtypes: int64(9), object(6)
memory usage: 123.4+ KB


In [20]:
course_data =data.read_table("mdl_course")
course_data.head()

,id,category,sortorder,fullname,shortname,idnumber,summary,summaryformat,format,showgrades,...,defaultgroupingid,lang,calendartype,theme,timecreated,timemodified,requested,enablecompletion,completionnotify,cacherev
0,7,6,60002,FDP Design -WordPress,FDP WordPress,,"<p><img src=""@@PLUGINFILE@@/WordPress.svg.png""...",1,topics,1,...,0,,,,1528706177,1548697122,0,1,0,1561503107
1,10,1,30001,How to design and facilitate an online course,Design eCourse,,"<p dir=""ltr""><span>After completing this cours...",1,topics,1,...,0,en,,boost,1374053678,1562773717,0,0,0,1562773718
2,50,10,80001,How data can impact development,Data for Development,,<p>This course is part of the Changemaking tra...,1,topics,1,...,0,en,,boost,1562232960,1562307278,0,1,0,1563791465
3,8,1,30002,Game-based Learning,Game and Learning,,"<p><p>In this module, you will explore the var...",1,topics,1,...,0,,,boost,1543612197,1562773782,0,0,0,1562773785
4,46,7,10001,Welcome to UpSkill - start here!,Welcome to UpSkill,,<p>Start here to learn about 10 Academy and wh...,1,topics,1,...,0,en,,boost,1550050414,1562772944,0,1,0,1562930608


In [21]:
course_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 31 columns):
id                   15 non-null int64
category             15 non-null int64
sortorder            15 non-null int64
fullname             15 non-null object
shortname            15 non-null object
idnumber             15 non-null object
summary              15 non-null object
summaryformat        15 non-null int64
format               15 non-null object
showgrades           15 non-null int64
newsitems            15 non-null int64
startdate            15 non-null int64
enddate              15 non-null int64
marker               15 non-null int64
maxbytes             15 non-null int64
legacyfiles          15 non-null int64
showreports          15 non-null int64
visible              15 non-null int64
visibleold           15 non-null int64
groupmode            15 non-null int64
groupmodeforce       15 non-null int64
defaultgroupingid    15 non-null int64
lang                 15 non-null o

In [22]:
module_data = data.read_table("mdl_modules")
module_data.head()

,id,name,cron,lastcron,search,visible
0,2,assignment,60,0,,0
1,3,book,0,0,,1
2,5,choice,0,0,,1
3,6,data,0,0,,1
4,7,feedback,0,0,,1


In [23]:
module_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 6 columns):
id          26 non-null int64
name        26 non-null object
cron        26 non-null int64
lastcron    26 non-null int64
search      26 non-null object
visible     26 non-null int64
dtypes: int64(4), object(2)
memory usage: 1.3+ KB


In [24]:
course_modules_data = data.read_table("mdl_course_modules")
course_modules_data.head()

,id,course,module,instance,section,idnumber,added,score,indent,visible,...,visibleold,groupmode,groupingid,completion,completiongradeitemnumber,completionview,completionexpected,showdescription,availability,deletioninprogress
0,1,1,9,1,2,None,1548435669,0,0,1,...,1,0,0,0,NaN,0,0,0,None,0
1,23,6,12,1,17,None,1206338932,0,0,1,...,1,0,0,0,NaN,0,0,0,None,0
2,69,8,15,2,29,,1543620947,0,0,1,...,1,0,0,0,NaN,0,0,0,None,0
3,24,6,9,4,17,None,1206338989,0,0,1,...,1,1,0,0,NaN,0,0,0,None,0
4,227,11,9,24,57,None,1549462268,0,0,1,...,1,0,0,0,NaN,0,0,0,None,0


In [25]:
course_modules_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 290 entries, 0 to 289
Data columns (total 21 columns):
id                           290 non-null int64
course                       290 non-null int64
module                       290 non-null int64
instance                     290 non-null int64
section                      290 non-null int64
idnumber                     127 non-null object
added                        290 non-null int64
score                        290 non-null int64
indent                       290 non-null int64
visible                      290 non-null int64
visibleoncoursepage          290 non-null int64
visibleold                   290 non-null int64
groupmode                    290 non-null int64
groupingid                   290 non-null int64
completion                   290 non-null int64
completiongradeitemnumber    24 non-null float64
completionview               290 non-null int64
completionexpected           290 non-null int64
showdescription              

In [26]:
course_modules_completion = data.read_table("mdl_course_modules_completion")
course_modules_completion.head()

,id,coursemoduleid,userid,completionstate,viewed,overrideby,timemodified
0,5801,224,56,1,0.0,None,1561988963
1,3,228,3,0,1.0,None,0
2,36,228,2,0,1.0,None,0
3,37,228,5,0,1.0,None,0
4,38,228,32,0,1.0,None,0


In [27]:
course_modules_completion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4483 entries, 0 to 4482
Data columns (total 7 columns):
id                 4483 non-null int64
coursemoduleid     4483 non-null int64
userid             4483 non-null int64
completionstate    4483 non-null int64
viewed             3466 non-null float64
overrideby         0 non-null object
timemodified       4483 non-null int64
dtypes: float64(1), int64(5), object(1)
memory usage: 245.2+ KB


In [28]:
grades_data = data.read_table("mdl_grade_grades")
grades_data.head()

,id,itemid,userid,rawgrade,rawgrademax,rawgrademin,rawscaleid,usermodified,finalgrade,hidden,...,overridden,excluded,feedback,feedbackformat,information,informationformat,timecreated,timemodified,aggregationstatus,aggregationweight
0,2,92,3,NaN,100.0,0.0,NaN,NaN,NaN,0,...,0,0,None,0,None,0,NaN,NaN,unknown,NaN
1,1,93,3,NaN,10.0,0.0,NaN,3.0,NaN,0,...,0,0,None,0,None,0,NaN,NaN,novalue,0.0
2,51,126,50,NaN,12.0,0.0,NaN,NaN,12.0,0,...,0,0,None,0,None,0,NaN,1.549723e+09,unknown,NaN
3,35,92,65,NaN,12.0,0.0,NaN,NaN,0.0,0,...,0,0,None,0,None,0,NaN,1.549642e+09,unknown,NaN
4,34,93,65,0.0,12.0,0.0,NaN,3.0,0.0,0,...,0,0,None,0,None,0,1.549640e+09,1.549642e+09,used,1.0


In [29]:
grades_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3643 entries, 0 to 3642
Data columns (total 23 columns):
id                   3643 non-null int64
itemid               3643 non-null int64
userid               3643 non-null int64
rawgrade             1089 non-null float64
rawgrademax          3643 non-null float64
rawgrademin          3643 non-null float64
rawscaleid           148 non-null float64
usermodified         1529 non-null float64
finalgrade           2160 non-null float64
hidden               3643 non-null int64
locked               3643 non-null int64
locktime             3643 non-null int64
exported             3643 non-null int64
overridden           3643 non-null int64
excluded             3643 non-null int64
feedback             4 non-null object
feedbackformat       3643 non-null int64
information          0 non-null object
informationformat    3643 non-null int64
timecreated          843 non-null float64
timemodified         2160 non-null float64
aggregationstatus    3

### Number of table in our database

# Number of Rows in Select Tables

## Number of quiz submissions by hour of day

In [30]:
%%sql
SELECT 
EXTRACT(HOUR FROM to_timestamp(mdl_logstore_standard_log.timecreated)) AS HOUR,
COUNT(*) FROM mdl_logstore_standard_log
WHERE (mdl_logstore_standard_log.action = 'submitted')
GROUP BY EXTRACT(HOUR FROM to_timestamp(mdl_logstore_standard_log.timecreated))
ORDER BY HOUR DESC;

UsageError: Cell magic `%%sql` not found.


## Monthly usage time of learners who have confirmed and are not deleted

In [31]:
%%sql
SELECT 
mdl_user.id AS User_Id,
EXTRACT(MONTH FROM to_timestamp(mdl_user.lastaccess)) - EXTRACT(MONTH FROM to_timestamp(mdl_user.firstaccess)) As monthly_usage_time
FROM mdl_user
WHERE (mdl_user.confirmed = 1)
AND (mdl_user.deleted = 0)
ORDER BY monthly_usage_time DESC;

UsageError: Cell magic `%%sql` not found.


## Count of log events per user for the following verbs: ['loggedin', 'viewed', 'started', ,'submitted', 'uploaded', 'updated', 'searched', 'resumed', 'answered', 'attempted', 'abandoned']

In [32]:
%%sql
SELECT mdl_logstore_standard_log.userid,
    SUM(CASE WHEN (action='loggedin') THEN userid  ELSE NULL END) AS logged_in,
    SUM(CASE WHEN (action='viewed') THEN userid  ELSE NULL END) AS viewed,
    SUM(CASE WHEN (action='started') THEN userid  ELSE NULL END) AS started,
    SUM(CASE WHEN (action='submitted') THEN userid  ELSE NULL END) AS submitted,
    SUM(CASE WHEN (action='uploaded') THEN userid  ELSE NULL END) AS uploaded,
    SUM(CASE WHEN (action='updated') THEN userid  ELSE NULL END) AS updated,
    SUM(CASE WHEN (action='searched') THEN userid  ELSE NULL END) AS searched,
    SUM(CASE WHEN (action='resumed') THEN userid  ELSE NULL END) AS resumed,
    SUM(CASE WHEN (action='answered') THEN userid  ELSE NULL END) AS answered,
    SUM(CASE WHEN (action='attempted') THEN userid  ELSE NULL END) AS attempted,
    SUM(CASE WHEN (action='abandoned') THEN userid  ELSE NULL END) AS abandoned
FROM mdl_logstore_standard_log
GROUP BY mdl_logstore_standard_log.userid

UsageError: Cell magic `%%sql` not found.


Python class to pull 
* Overall grade of learners for 
* Number of forum posts


In [33]:
engine = create_engine('postgresql://postgres:qwerty111!@@localhost/moodle')

In [34]:
#!pip install pandas

In [ ]:
"""mdl_logstore_standard_log
mdl_context
mdl_user
mdl_course
mdl_modules 
mdl_course_modules
mdl_course_modules_completion 
mdl_grade_grades"""